In [1]:
import semantic_kernel as sk
import logging
import os
from dotenv import load_dotenv, find_dotenv
from helpers.LoggingHelper import LoggingHelper

_ = load_dotenv(find_dotenv()) # read local .env file
print(os.getenv("OPENAI_API_BASE"))

lh = LoggingHelper()
logger = lh.get_logger("semantic-kernel", logging.WARN)


https://alkopenai2.openai.azure.com/


In [2]:
from pprint import pprint
import semantic_kernel.connectors.ai.open_ai as sk_oai

kernel = sk.Kernel(log=logger)
kernel.add_chat_service(
    "chat_completion",
    sk_oai.AzureChatCompletion(
        "GPT35",
        os.getenv("OPENAI_API_BASE"),
        os.getenv("OPENAI_API_KEY")
    ),
)

# Register a volatile memory store
# pprint(dir(kernel))
kernel.add_text_embedding_generation_service(
    "ada", 
    sk_oai.OpenAITextEmbedding("text-embedding-ada-002", "api_key", "org_id"))
kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())

In [3]:
# Now we need to import the plugin
from plugins.JarvisPlugin.Tasks import Tasks

# Now you can import the plugin importing skill directly from the function you declared
# in the plugin directory. The import_skill does not need the path, it only need an
# instance of the skill and the name of the skill
tasks_plugin = kernel.import_skill(Tasks(), skill_name="JarvisPlugin")


In [4]:
from pprint import pprint
# want to print all the keys of extractaudio_plugin that is a dictionary

pprint (tasks_plugin["ChangeTaskTitle"])
pprint (tasks_plugin["LoadTask"])

In [5]:
# We start using a planner, that allows us not to have to manually call functions
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [6]:
# # now we can use the planner
# Task is too complex for a simple epaln
# question = "Search all task containing 'verdolazzy' and if the due date is in year 2021 add the word 'BOMB' as prefix of the title"
# plan = await planner.create_plan_async(question, kernel)

# result = await planner.execute_plan_async(plan, kernel)

# print(f"Plan results: {result}")

# print("------------------")
# print(plan.generated_plan)

In [7]:
# now we can create a stepwise planner
from semantic_kernel.planning.stepwise_planner import StepwisePlanner
sw_planner = StepwisePlanner(kernel)

In [9]:
from pprint import pprint

question = "Search all task containing verdolazzy and if the due date is in year 2021 add the word 'BOMB' as prefix of the title"
plan = sw_planner.create_plan(question)
result = await plan.invoke_async()

print(f"Plan results: {result}")
pprint(result)


2023-11-30 19:50:17,416 - semantic-kernel - WARNING - Variable `$agent_scratch_pad` not found


Searching for task: 


CancelledError: 